In [ ]:
import cfgrib
import xarray as xr

In [ ]:
data = xr.open_dataset("Z_NAFP_C_BABJ_20250818000000_P_CMA-GEPS-GLB-036.grib2", engine='cfgrib',decode_timedelta=True )
data

In [ ]:
print("数据维度:", data.dims)
print("数据变量:", list(data.data_vars))

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
# grib api

import gribapi
from gribapi import *

# eccodes

import eccodes
from eccodes import *


In [ ]:
import xarray as xr

file_path = "Z_NAFP_C_BABJ_20250818000000_P_CMA-GEPS-GLB-036.grib2"  # 修正文件名（如果有误）

# 尝试读取特定变量（例如 'ta'）
ds = xr.open_dataset(
    file_path,
    engine='cfgrib',
    filter_by_keys={'shortName': 'ta'}  # 只读取 'ta' 变量
)

print(ds)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

file_path = "Z_NAFP_C_BABJ_20250818000000_P_CMA-GEPS-GLB-036.grib2"

try:
    # 尝试使用 cfgrib 读取数据，仅使用合法参数
    ds = xr.open_dataset(
        file_path,
        engine='cfgrib',
        backend_kwargs={
            'filter_by_keys': {'shortName': 'ta'},  # 只读取 'ta' 变量
            'read_keys': ['edition', 'typeOfLevel'],  # 显式指定读取的键（可选）
        }
    )
    print("数据加载成功！")
    print(ds)
except Exception as e:
    print(f"cfgrib 加载失败: {e}")
    try:
        # 备选方案：使用 pygrib
        print("尝试使用 pygrib 引擎...")
        ds = xr.open_dataset(file_path, engine='pygrib')
        print("使用 pygrib 加载成功！")
    except Exception as e2:
        print(f"pygrib 加载失败: {e2}")
        raise RuntimeError("无法加载 GRIB 文件，请检查文件或环境配置。")

# 提取变量并绘图
ta = ds['ta']

# 创建绘图
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())  # 使用 PlateCarree 投影

# 检查数据维度并处理
if 'longitude' not in ds.coords or 'latitude' not in ds.coords:
    # 尝试查找可能的经纬度坐标变量
    lon_names = ['lon', 'longitude', 'longitudes']
    lat_names = ['lat', 'latitude', 'latitudes']
    lon_var = next((name for name in lon_names if name in ds.variables), None)
    lat_var = next((name for name in lat_names if name in ds.variables), None)
    
    if lon_var and lat_var:
        lons = ds[lon_var]
        lats = ds[lat_var]
    else:
        raise ValueError("无法找到经纬度坐标变量")
else:
    lons = ds['longitude']
    lats = ds['latitude']

# 绘制填充等高线图
contour = ax.contourf(
    lons,  # 经度
    lats,   # 纬度
    ta,  # 直接使用二维数据
    levels=20,        # 等高线层级数
    cmap='coolwarm',  # 颜色映射
    transform=ccrs.PlateCarree()  # 数据坐标系
)

# 添加地理特征
ax.coastlines(resolution='110m', color='black')  # 海岸线
ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgray')  # 陆地
ax.add_feature(cfeature.OCEAN, edgecolor='none', facecolor='white')  # 海洋
ax.set_title("Global 2m Temperature (ta)", fontsize=14)  # 标题

# 添加颜色条
cbar = fig.colorbar(contour, ax=ax, orientation='horizontal', pad=0.05, aspect=30)
cbar.set_label("Temperature (K)", fontsize=12)  # 颜色条标签

# 显示图像
plt.tight_layout()
plt.show()

In [ ]:
from __future__ import print_function
import sys
import eccodes

def process_grib(file_path):
    try:
        with open(file_path, 'rb') as f:
            while True:
                handle = eccodes.codes_grib_new_from_file(f, headers_only=False)
                if handle is None:
                    break

                # 提取关键信息
                short_name = eccodes.codes_get_string(handle, "shortName")
                param_id = eccodes.codes_get_long(handle, "paramId")
                level_type = eccodes.codes_get_string(handle, "typeOfLevel")
                level = eccodes.codes_get_long(handle, "level")
                data_date = eccodes.codes_get_string(handle, "dataDate")
                data_time = eccodes.codes_get_string(handle, "dataTime")

                print(f"Variable: {short_name} (paramId={param_id})")
                print(f"  Level: {level} hPa ({level_type})")
                print(f"  Time: {data_date} {data_time} UTC")
                print("-" * 40)

                eccodes.codes_release(handle)

    except Exception as e:
        print(f"ERROR: {str(e)}", file=sys.stderr)

# 直接调用函数，传入 GRIB2 文件路径
process_grib("Z_NAFP_C_BABJ_20250818000000_P_CMA-GEPS-GLB-036.grib2")